In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *
from IPython.display import display,HTML

# NLP Deep Dive: RNNs

In <<chapter_intro>> we saw that deep learning can be used to get great results with natural language datasets. Our example relied on using a pretrained language model and fine-tuning it to classify reviews. That example highlighted a difference between transfer learning in NLP and computer vision: in general in NLP the pretrained model is trained on a different task.

What we call a language model is a model that has been trained to guess what the next word in a text is (having read the ones before). This kind of task is called *self-supervised learning*: we do not need to give labels to our model, just feed it lots and lots of texts. It has a process to automatically get labels from the data, and this task isn't trivial: to properly guess the next word in a sentence, the model will have to develop an understanding of the English (or other) language. Self-supervised learning can also be used in other domains; for instance, see ["Self-Supervised Learning and Computer Vision"](https://www.fast.ai/2020/01/13/self_supervised/) for an introduction to vision applications. Self-supervised learning is not usually used for the model that is trained directly, but instead is used for pretraining a model used for transfer learning.

---

在 <<chapter_intro>> 中，我们看到深度学习可用于在自然语言数据集上获得出色的结果。我们的示例依赖于使用预训练的语言模型并对其进行微调来对评论进行分类。这个例子强调了 NLP 中的迁移学习与计算机视觉之间的区别：一般来说，NLP 中的预训练模型是在不同的任务中进行训练的。

我们所说的语言模型是一个经过训练的模型，它可以猜测文本中的下一个单词是什么（在阅读了前面的单词之后）。这种任务被称为 *自我监督学习* ：我们不需要给我们的模型贴标签，只需要给它提供大量的文本。它有一个从数据中自动获取标签的过程，而这项任务并不简单：要正确猜出句子中的下一个单词，模型必须对英语（或其他）语言有一定的理解。自监督学习也可用于其他领域，例如，请参阅["自监督学习与计算机视觉"](https://www.fast.ai/2020/01/13/self_supervised/)，了解有关视觉应用的介绍。自我监督学习通常不用于直接训练模型，而是用于预训练用于迁移学习的模型。

> jargon: Self-supervised learning: Training a model using labels that are embedded in the independent variable, rather than requiring external labels. For instance, training a model to predict the next word in a text.

---

> 行话：自我监督学习：使用嵌入在自变量中的标签来训练模型，而不需要外部标签。例如，训练一个模型来预测文本中的下一个单词。

The language model we used in <<chapter_intro>> to classify IMDb reviews was pretrained on Wikipedia. We got great results by directly fine-tuning this language model to a movie review classifier, but with one extra step, we can do even better. The Wikipedia English is slightly different from the IMDb English, so instead of jumping directly to the classifier, we could fine-tune our pretrained language model to the IMDb corpus and then use *that* as the base for our classifier.

Even if our language model knows the basics of the language we are using in the task (e.g., our pretrained model is in English), it helps to get used to the style of the corpus we are targeting. It may be more informal language, or more technical, with new words to learn or different ways of composing sentences. In the case of the IMDb dataset, there will be lots of names of movie directors and actors, and often a less formal style of language than that seen in Wikipedia.

We already saw that with fastai, we can download a pretrained English language model and use it to get state-of-the-art results for NLP classification. (We expect pretrained models in many more languages to be available soon—they might well be available by the time you are reading this book, in fact.) So, why are we learning how to train a language model in detail?

One reason, of course, is that it is helpful to understand the foundations of the models that you are using. But there is another very practical reason, which is that you get even better results if you fine-tune the (sequence-based) language model prior to fine-tuning the classification model. For instance, for the IMDb sentiment analysis task, the dataset includes 50,000 additional movie reviews that do not have any positive or negative labels attached. Since there are 25,000 labeled reviews in the training set and 25,000 in the validation set, that makes 100,000 movie reviews altogether. We can use all of these reviews to fine-tune the pretrained language model, which was trained only on Wikipedia articles; this will result in a language model that is particularly good at predicting the next word of a movie review.

This is known as the Universal Language Model Fine-tuning (ULMFit) approach. The [paper](https://arxiv.org/abs/1801.06146) showed that this extra stage of fine-tuning of the language model, prior to transfer learning to a classification task, resulted in significantly better predictions. Using this approach, we have three stages for transfer learning in NLP, as summarized in <<ulmfit_process>>.

---

我们在 <<chapter_intro>> 中用于对 IMDb 评论进行分类的语言模型是在维基百科上进行预训练的。我们将该语言模型直接微调为电影评论分类器，取得了很好的效果，但只要多做一步，我们就能做得更好。维基百科的英语与 IMDb 的英语略有不同，因此我们可以根据 IMDb 的语料库微调预训练的语言模型，然后将其作为分类器的基础，而不是直接跳转到分类器。

即使我们的语言模型知道我们在任务中使用的语言的基础知识（例如，我们的预训练模型使用英语），习惯我们所针对的语料库的风格也是有帮助的。它可能是更非正式的语言，也可能是更专业的语言，需要学习新的单词或不同的造句方式。在 IMDb 数据集中，会有很多电影导演和演员的名字，语言风格通常没有维基百科中那么正式。

我们已经看到，通过 fastai，我们可以下载一个经过预训练的英语语言模型，并利用它获得最先进的 NLP 分类结果。(我们期待更多语言的预训练模型很快就会问世--事实上，在你阅读本书的时候，它们可能已经问世了）。那么，我们为什么要详细学习如何训练语言模型呢？

当然，其中一个原因是，了解所使用模型的基础很有帮助。但还有一个非常实用的原因，那就是如果在微调分类模型之前微调（基于序列的）语言模型，就能获得更好的结果。例如，在 IMDb 的情感分析任务中，数据集包括 50,000 篇没有附加任何正面或负面标签的额外电影评论。由于训练集中有 25,000 篇带标签的评论，验证集中有 25,000 篇带标签的评论，因此总共有 100,000 篇电影评论。我们可以使用所有这些影评来微调预训练语言模型，该模型只在维基百科文章中进行过训练；这将产生一个特别擅长预测影评下一个单词的语言模型。

这就是所谓的通用语言模型微调（ULMFit）方法。论文](https://arxiv.org/abs/1801.06146)表明，在将学习转移到分类任务之前，对语言模型进行额外的微调，能显著提高预测效果。使用这种方法，我们可以在 NLP 中进行三个阶段的迁移学习，如 <<ulmfit_process>> 所总结的那样。

<img alt="Diagram of the ULMFiT process" width="700" caption="The ULMFiT process" id="ulmfit_process" src="images/att_00027.png">

We'll now explore how to apply a neural network to this language modeling problem, using the concepts introduced in the last two chapters. But before reading further, pause and think about how *you* would approach this.

---

现在，我们将利用前两章介绍的概念，探讨如何将神经网络应用于这一语言建模问题。但在继续阅读之前，请先暂停一下，想一想*你*会如何处理这个问题。

## Text Preprocessing 文本预处理

It's not at all obvious how we're going to use what we've learned so far to build a language model. Sentences can be different lengths, and documents can be very long. So, how can we predict the next word of a sentence using a neural network? Let's find out!

We've already seen how categorical variables can be used as independent variables for a neural network. The approach we took for a single categorical variable was to:

1. Make a list of all possible levels of that categorical variable (we'll call this list the *vocab*).
1. Replace each level with its index in the vocab.
1. Create an embedding matrix for this containing a row for each level (i.e., for each item of the vocab).
1. Use this embedding matrix as the first layer of a neural network. (A dedicated embedding matrix can take as inputs the raw vocab indexes created in step 2; this is equivalent to but faster and more efficient than a matrix that takes as input one-hot-encoded vectors representing the indexes.)

We can do nearly the same thing with text! What is new is the idea of a sequence. First we concatenate all of the documents in our dataset into one big long string and split it into words, giving us a very long list of words (or "tokens"). Our independent variable will be the sequence of words starting with the first word in our very long list and ending with the second to last, and our dependent variable will be the sequence of words starting with the second word and ending with the last word. 

Our vocab will consist of a mix of common words that are already in the vocabulary of our pretrained model and new words specific to our corpus (cinematographic terms or actors names, for instance). Our embedding matrix will be built accordingly: for words that are in the vocabulary of our pretrained model, we will take the corresponding row in the embedding matrix of the pretrained model; but for new words we won't have anything, so we will just initialize the corresponding row with a random vector.

---

目前我们还不清楚将如何使用迄今为止所学到的知识来构建语言模型。句子可以有不同的长度，文档可以很长。那么，我们如何使用神经网络预测句子的下一个单词呢？让我们来了解一下吧！

我们已经看到了如何将分类变量用作神经网络的自变量。我们对单个分类变量采取的方法是：

1. 列出该分类变量的所有可能级别（我们将此列表称为 *vocab*）。
2. 将每个级别替换为词汇表中的索引。
3. 为此创建一个嵌入矩阵，其中包含每个级别（即词汇表的每个项目）的一行。
4. 使用此嵌入矩阵作为神经网络的第一层。（专用嵌入矩阵可以将步骤 2 中创建的原始词汇索引作为输入;这相当于将表示索引的 one-hot 编码向量作为输入的矩阵，但速度更快、效率更高。

我们可以用文本做几乎同样的事情！新的是序列的概念。首先，我们将数据集中的所有文档连接成一个大的长字符串，并将其拆分为单词，从而为我们提供了一个很长的单词（或“标记”）列表。我们的自变量是从我们很长的列表中的第一个单词开始，到倒数第二个单词结束的单词序列，我们的因变量是从第二个单词开始，到最后一个单词结束的单词序列。

我们的词汇表将包括预训练模型词汇表中已有的常用词和语料库特有的新词（例如电影术语或演员名称）。我们的嵌入矩阵将相应地构建：对于预训练模型词汇表中的单词，我们将在预训练模型的嵌入矩阵中取相应的行;但是对于新单词，我们将什么都没有，所以我们只需使用随机向量初始化相应的行。

Each of the steps necessary to create a language model has jargon associated with it from the world of natural language processing, and fastai and PyTorch classes available to help. The steps are:

- Tokenization:: Convert the text into a list of words (or characters, or substrings, depending on the granularity of your model)
- Numericalization:: Make a list of all of the unique words that appear (the vocab), and convert each word into a number, by looking up its index in the vocab
- Language model data loader creation:: fastai provides an `LMDataLoader` class which automatically handles creating a dependent variable that is offset from the independent variable by one token. It also handles some important details, such as how to shuffle the training data in such a way that the dependent and independent variables maintain their structure as required
- Language model creation:: We need a special kind of model that does something we haven't seen before: handles input lists which could be arbitrarily big or small. There are a number of ways to do this; in this chapter we will be using a *recurrent neural network* (RNN). We will get to the details of these RNNs in the <<chapter_nlp_dive>>, but for now, you can think of it as just another deep neural network.

Let's take a look at how each step works in detail.

---

创建语言模型所需的每个步骤都与自然语言处理领域的专业术语有关，也有 fastai 和 PyTorch 类可以提供帮助。这些步骤是

- 标记化（Tokenization）：：将文本转换为单词列表（或字符或子串，取决于模型的粒度）
- 数值化（Numericalization）：：列出所有出现的唯一单词（词汇表），并通过查找词汇表中的索引将每个单词转换成数字
- 语言模型数据加载器的创建：fastai 提供了一个 `LMDataLoader` 类，可自动处理因变量的创建，该因变量与自变量之间有一个标记的偏移。它还会处理一些重要的细节，例如如何以因变量和自变量按要求保持结构的方式洗牌训练数据。
- 创建语言模型：：我们需要一种特殊的模型，它能做一些我们以前从未见过的事情：处理可大可小的输入列表。有许多方法可以做到这一点；在本章中，我们将使用*递归神经网络*（RNN）。我们将在<<chapter_nlp_dive>>中详细介绍这些 RNN，但现在，你可以将其视为另一种深度神经网络。

让我们来看看每个步骤的详细工作原理。

### Tokenization

When we said "convert the text into a list of words," we left out a lot of details. For instance, what do we do with punctuation? How do we deal with a word like "don't"? Is it one word, or two? What about long medical or chemical words? Should they be split into their separate pieces of meaning? How about hyphenated words? What about languages like German and Polish where we can create really long words from many, many pieces? What about languages like Japanese and Chinese that don't use bases at all, and don't really have a well-defined idea of *word*?

Because there is no one correct answer to these questions, there is no one approach to tokenization. There are three main approaches:

- Word-based:: Split a sentence on spaces, as well as applying language-specific rules to try to separate parts of meaning even when there are no spaces (such as turning "don't" into "do n't"). Generally, punctuation marks are also split into separate tokens.
- Subword based:: Split words into smaller parts, based on the most commonly occurring substrings. For instance, "occasion" might be tokenized as "o c ca sion."
- Character-based:: Split a sentence into its individual characters.

We'll be looking at word and subword tokenization here, and we'll leave character-based tokenization for you to implement in the questionnaire at the end of this chapter.

---

当我们说 "将文本转换成单词表" 时，我们遗漏了很多细节。例如，我们如何处理标点符号？如何处理 "don't "这样的单词？是一个词，还是两个词？长的医学或化学单词怎么办？是否应该将它们拆分成各自独立的含义？连字符词如何处理？在德语和波兰语等语言中，我们可以用很多很多片段创造出很长的单词，那么德语和波兰语呢？那么像日语和汉语这样根本不使用词基，也没有明确定义的 *词* 概念的语言呢？

因为这些问题没有一个正确的答案，所以也就没有一种标记化方法。主要有三种方法：

- 基于单词：：根据空格分割句子，并应用特定的语言规则，即使在没有空格的情况下，也要尝试分割意义部分（例如将 "don't "转化为 "do n't"）。一般来说，标点符号也会被分割成单独的标记。
- 基于子词：：根据最常出现的子串将单词拆分成更小的部分。例如，"occasion" 可以标记为 "o c ca sion"。
- 基于字符：：将句子分割成单个字符。

我们将在这里介绍单词和子词标记化，基于字符的标记化将留待您在本章结尾的问卷中实现。

> jargon: token: One element of a list created by the tokenization process. It could be a word, part of a word (a _subword_), or a single character.

---

> 行话：标记：标记化过程创建的列表中的一个元素。它可以是一个词、词的一部分（_子词_）或单个字符。

### Word Tokenization with fastai

Rather than providing its own tokenizers, fastai instead provides a consistent interface to a range of tokenizers in external libraries. Tokenization is an active field of research, and new and improved tokenizers are coming out all the time, so the defaults that fastai uses change too. However, the API and options shouldn't change too much, since fastai tries to maintain a consistent API even as the underlying technology changes.

Let's try it out with the IMDb dataset that we used in <<chapter_intro>>:

---

fastai 并没有提供自己的标记化器，而是为外部库中的一系列标记化器提供了一个一致的接口。标记化是一个活跃的研究领域，新的和改进的标记化器层出不穷，因此 fastai 使用的默认值也在不断变化。不过，应用程序接口和选项应该不会有太大变化，因为 fastai 试图在底层技术发生变化时保持一致的应用程序接口。

让我们用 <<chapter_intro>> 中使用的 IMDb 数据集试试看：

In [ ]:
from fastai.text.all import *
path = untar_data(URLs.IMDB)

We'll need to grab the text files in order to try out a tokenizer. Just like `get_image_files`, which we've used many times already, gets all the image files in a path, `get_text_files` gets all the text files in a path. We can also optionally pass `folders` to restrict the search to a particular list of subfolders:

---

我们需要抓取文本文件，以便试用标记符。就像我们已经用过很多次的 `get_image_files` 获取路径中的所有图像文件一样，`get_text_files` 获取路径中的所有文本文件。我们还可以选择传递 `folders` 来将搜索限制在特定的子文件夹列表中：

In [ ]:
files = get_text_files(path, folders = ['train', 'test', 'unsup'])

Here's a review that we'll tokenize (we'll just print the start of it here to save space):

---

下面是一篇我们将标记化的评论（为节省篇幅，我们只打印开头部分）：

In [ ]:
txt = files[0].open().read(); txt[:75]

'This movie, which I just discovered at the video store, has apparently sit '

As we write this book, the default English word tokenizer for fastai uses a library called *spaCy*. It has a sophisticated rules engine with special rules for URLs, individual special English words, and much more. Rather than directly using `SpacyTokenizer`, however, we'll use `WordTokenizer`, since that will always point to fastai's current default word tokenizer (which may not necessarily be spaCy, depending when you're reading this).

Let's try it out. We'll use fastai's `coll_repr(collection, n)` function to display the results. This displays the first *`n`* items of *`collection`*, along with the full size—it's what `L` uses by default. Note that fastai's tokenizers take a collection of documents to tokenize, so we have to wrap `txt` in a list:

---

在我们编写本书时，fastai 的默认英文单词标记器使用了一个名为 *spaCy* 的库。它有一个复杂的规则引擎，包含针对 URL、个别特殊英文单词等的特殊规则。不过，我们不会直接使用 `SpacyTokenizer`，而是使用 `WordTokenizer`，因为它将始终指向 fastai 当前的默认单词标记符号生成器（不一定是 spaCy，这取决于您阅读本书的时间）。

让我们试试看。我们将使用 fastai 的 `coll_repr(collection, n)` 函数来显示结果。这会显示 *`collection`* 的前 *`n`* 项，以及完整的大小--这是 `L` 默认使用的。请注意，fastai 的标记符号化器需要一个文档集合来标记，因此我们必须用一个列表来封装 `txt`：

In [ ]:
spacy = WordTokenizer()
toks = first(spacy([txt]))
print(coll_repr(toks, 30))

(#201) ['This','movie',',','which','I','just','discovered','at','the','video','store',',','has','apparently','sit','around','for','a','couple','of','years','without','a','distributor','.','It',"'s",'easy','to','see'...]


As you see, spaCy has mainly just separated out the words and punctuation. But it does something else here too: it has split "it's" into "it" and "'s". That makes intuitive sense; these are separate words, really. Tokenization is a surprisingly subtle task, when you think about all the little details that have to be handled. Fortunately, spaCy handles these pretty well for us—for instance, here we see that "." is separated when it terminates a sentence, but not in an acronym or number:

---

如您所见，spaCy 主要是将单词和标点符号分开。但它还做了一件事：它将 "it's "分成了 "it "和"'s"。这很直观，因为这确实是两个独立的词。如果考虑到需要处理的所有小细节，标记化是一项令人惊讶的微妙工作。幸运的是，spaCy 可以很好地处理这些细节--例如，在这里我们可以看到，". "在结束句子时会被分开，但在缩写词或数字中则不会：

In [ ]:
first(spacy(['The U.S. dollar $1 is $1.00.']))

(#9) ['The','U.S.','dollar','$','1','is','$','1.00','.']

fastai then adds some additional functionality to the tokenization process with the `Tokenizer` class:

---

fastai 还为 `Tokenizer` 类的标记化过程添加了一些附加功能：

In [ ]:
tkn = Tokenizer(spacy)
print(coll_repr(tkn(txt), 31))

(#228) ['xxbos','xxmaj','this','movie',',','which','i','just','discovered','at','the','video','store',',','has','apparently','sit','around','for','a','couple','of','years','without','a','distributor','.','xxmaj','it',"'s",'easy'...]


Notice that there are now some tokens that start with the characters "xx", which is not a common word prefix in English. These are *special tokens*.

For example, the first item in the list, `xxbos`, is a special token that indicates the start of a new text ("BOS" is a standard NLP acronym that means "beginning of stream"). By recognizing this start token, the model will be able to learn it needs to "forget" what was said previously and focus on upcoming words.

These special tokens don't come from spaCy directly. They are there because fastai adds them by default, by applying a number of rules when processing text. These rules are designed to make it easier for a model to recognize the important parts of a sentence. In a sense, we are translating the original English language sequence into a simplified tokenized language—a language that is designed to be easy for a model to learn.

For instance, the rules will replace a sequence of four exclamation points with a special *repeated character* token, followed by the number four, and then a single exclamation point. In this way, the model's embedding matrix can encode information about general concepts such as repeated punctuation rather than requiring a separate token for every number of repetitions of every punctuation mark. Similarly, a capitalized word will be replaced with a special capitalization token, followed by the lowercase version of the word. This way, the embedding matrix only needs the lowercase versions of the words, saving compute and memory resources, but can still learn the concept of capitalization.

Here are some of the main special tokens you'll see:

- `xxbos`:: Indicates the beginning of a text (here, a review)
- `xxmaj`:: Indicates the next word begins with a capital (since we lowercased everything)
- `xxunk`:: Indicates the word is unknown

To see the rules that were used, you can check the default rules:

---

请注意，现在有一些标记以 "xx" 开头，而它在英语中并不是常见的单词前缀。这些是 *特殊标记* 。

例如，列表中的第一个项目 `xxbos` 就是一个特殊标记，表示新文本的开始（"BOS" 是一个标准的 NLP 首字母缩写，意思是 `流的开始（"beginning of stream"）`）。通过识别这个起始标记，模型将能够学习到它需要 "忘记" 之前说过的话，并将注意力集中在即将出现的词语上。

这些特殊标记并非直接来自 spaCy。它们之所以存在，是因为 fastai 在处理文本时应用了一系列规则，在默认情况下添加了它们。这些规则旨在让模型更容易识别句子的重要部分。从某种意义上说，我们是在将原始的英语语序翻译成一种简化的标记化语言--一种旨在让模型更容易学习的语言。

例如，规则会用一个特殊的 *重复字符* 标记来替换一连串的四个感叹号，接着是数字四，然后是一个感叹号。这样，模型的嵌入矩阵就可以编码重复标点符号等一般概念的信息，而不需要为每一个标点符号的重复次数单独设置一个标记。同样，一个大写单词将被一个特殊的大写标记所替换，然后是该单词的小写版本。这样，嵌入矩阵只需要单词的小写版本，从而节省了计算和内存资源，但仍然可以学习大写的概念。

下面是你会看到的一些主要的特殊标记：

- `xxbos`:：表示文本的开头（这里是评论）
- `xxmaj`:：表示下一个单词以大写字母开头（因为我们把所有单词都小写了）
- `xxunk`:：表示单词未知

要查看使用的规则，可以查看默认规则：

In [ ]:
defaults.text_proc_rules

[<function fastai.text.core.fix_html(x)>,
 <function fastai.text.core.replace_rep(t)>,
 <function fastai.text.core.replace_wrep(t)>,
 <function fastai.text.core.spec_add_spaces(t)>,
 <function fastai.text.core.rm_useless_spaces(t)>,
 <function fastai.text.core.replace_all_caps(t)>,
 <function fastai.text.core.replace_maj(t)>,
 <function fastai.text.core.lowercase(t, add_bos=True, add_eos=False)>]

As always, you can look at the source code of each of them in a notebook by typing:

```
??replace_rep
```

Here is a brief summary of what each does:

- `fix_html`:: Replaces special HTML characters with a readable version (IMDb reviews have quite a few of these)
- `replace_rep`:: Replaces any character repeated three times or more with a special token for repetition (`xxrep`), the number of times it's repeated, then the character
- `replace_wrep`:: Replaces any word repeated three times or more with a special token for word repetition (`xxwrep`), the number of times it's repeated, then the word
- `spec_add_spaces`:: Adds spaces around / and #
- `rm_useless_spaces`:: Removes all repetitions of the space character
- `replace_all_caps`:: Lowercases a word written in all caps and adds a special token for all caps (`xxup`) in front of it
- `replace_maj`:: Lowercases a capitalized word and adds a special token for capitalized (`xxmaj`) in front of it
- `lowercase`:: Lowercases all text and adds a special token at the beginning (`xxbos`) and/or the end (`xxeos`)

---

与往常一样，您可以在笔记本上键入查看每个程序的源代码：

```
replace_rep
```

下面简要介绍了它们各自的作用：

- `fix_html`:：用可读版本替换特殊的 HTML 字符（IMDb 评论中有很多这样的字符）
- `replace_rep`:：用一个表示重复的特殊标记（`xxrep`）、重复的次数以及字符替换重复三次或三次以上的任何字符
- `replace_wrep`:：用一个用于表示单词重复的特殊符号（`xxwrep`）替换重复出现三次或三次以上的任何单词、重复出现的次数，然后是单词
- `spec_add_spaces`:：在 / 和 # 之间添加空格
- `rm_useless_spaces`:：删除所有重复的空格字符
- `replace_all_caps`:：小写全大写的单词，并在前面添加全大写的特殊标记 (`xxup`)
- `replace_maj`:：降低大写单词的大小写，并在前面添加表示大写的特殊标记 (`xxmaj`)
- `lowercase`:：小写所有文本，并在开头（`xxbos`）和/或结尾（`xxeos`）添加特殊标记

Let's take a look at a few of them in action:

---

让我们来看看它们中的几个例子：

In [ ]:
coll_repr(tkn('&copy;   Fast.ai www.fast.ai/INDEX'), 31)

"(#11) ['xxbos','©','xxmaj','fast.ai','xxrep','3','w','.fast.ai','/','xxup','index'...]"

Now let's take a look at how subword tokenization would work.

---

现在让我们来看看子词标记化是如何工作的。

### Subword Tokenization

In addition to the *word tokenization* approach seen in the last section, another popular tokenization method is *subword tokenization*. Word tokenization relies on an assumption that spaces provide a useful separation of components of meaning in a sentence. However, this assumption is not always appropriate. For instance, consider this sentence: 我的名字是郝杰瑞 ("My name is Jeremy Howard" in Chinese). That's not going to work very well with a word tokenizer, because there are no spaces in it! Languages like Chinese and Japanese don't use spaces, and in fact they don't even have a well-defined concept of a "word." There are also languages, like Turkish and Hungarian, that can add many subwords together without spaces, creating very long words that include a lot of separate pieces of information.

To handle these cases, it's generally best to use subword tokenization. This proceeds in two steps:

1. Analyze a corpus of documents to find the most commonly occurring groups of letters. These become the vocab.
2. Tokenize the corpus using this vocab of *subword units*.

Let's look at an example. For our corpus, we'll use the first 2,000 movie reviews:

---

除了上一节提到的 *词标记化* 方法外，另一种流行的标记化方法是 *子词标记化* 。单词标记化依赖于一种假设，即空格可以有效地分离句子中的意义成分。然而，这种假设并不总是合适的。例如，请看这个句子：我的名字是郝杰瑞。这在单词标记器中效果并不好，因为其中没有空格！汉语和日语等语言不使用空格，事实上，它们甚至没有定义明确的 "词" 的概念。还有一些语言，如土耳其语和匈牙利语，可以将许多子词加在一起而不留空格，从而创造出包含大量独立信息的超长单词。

要处理这些情况，通常最好使用子词标记化。这分两步进行：

1. 分析文档语料库，找出最常出现的字母群。这些就是词汇。
2. 使用这个 *子词单元* 的词汇表对语料库进行标记。

让我们来看一个例子。对于我们的语料库，我们将使用前 2,000 篇电影评论：

In [ ]:
txts = L(o.open().read() for o in files[:2000])

We instantiate our tokenizer, passing in the size of the vocab we want to create, and then we need to "train" it. That is, we need to have it read our documents and find the common sequences of characters to create the vocab. This is done with `setup`. As we'll see shortly, `setup` is a special fastai method that is called automatically in our usual data processing pipelines. Since we're doing everything manually at the moment, however, we have to call it ourselves. Here's a function that does these steps for a given vocab size, and shows an example output:

---

我们实例化我们的标记化器，传入我们要创建的词汇表的大小，然后我们需要 "训练" 它。也就是说，我们需要让它读取我们的文档，并找到共同的字符序列来创建词汇表。这是通过 `setup` 来完成的。我们很快就会看到，`setup` 是一个特殊的 fastai 方法，在我们通常的数据处理管道中会自动调用。不过，由于目前我们都是手动操作，所以我们必须自己调用它。下面的函数可以在给定词汇表大小的情况下完成这些步骤，并显示了一个输出示例：

In [ ]:
def subword(sz):
    sp = SubwordTokenizer(vocab_sz=sz)
    sp.setup(txts)
    return ' '.join(first(sp([txt]))[:40])

Let's try it out:

---

让我们试一试：

In [ ]:
subword(1000)

'▁This ▁movie , ▁which ▁I ▁just ▁dis c over ed ▁at ▁the ▁video ▁st or e , ▁has ▁a p par ent ly ▁s it ▁around ▁for ▁a ▁couple ▁of ▁years ▁without ▁a ▁dis t ri but or . ▁It'

When using fastai's subword tokenizer, the special character `▁` represents a space character in the original text.

If we use a smaller vocab, then each token will represent fewer characters, and it will take more tokens to represent a sentence:

---

在使用 fastai 的子字标记符时，特殊字符 `▁` 代表原始文本中的空格字符。

如果我们使用较小的词汇表，那么每个标记所代表的字符数就会减少，表示一个句子所需的标记数也会增加：

In [ ]:
subword(200)

'▁ T h i s ▁movie , ▁w h i ch ▁I ▁ j us t ▁ d i s c o ver ed ▁a t ▁the ▁ v id e o ▁ st or e , ▁h a s'

On the other hand, if we use a larger vocab, then most common English words will end up in the vocab themselves, and we will not need as many to represent a sentence:

---

另一方面，如果我们使用较大的词汇表，那么最常见的英语单词可能会包含在词汇表中，因此我们不需要像上面那样使用那么多来表示一个句子：

In [ ]:
subword(10000)

"▁This ▁movie , ▁which ▁I ▁just ▁discover ed ▁at ▁the ▁video ▁store , ▁has ▁apparently ▁sit ▁around ▁for ▁a ▁couple ▁of ▁years ▁without ▁a ▁distributor . ▁It ' s ▁easy ▁to ▁see ▁why . ▁The ▁story ▁of ▁two ▁friends ▁living"

Picking a subword vocab size represents a compromise: a larger vocab means fewer tokens per sentence, which means faster training, less memory, and less state for the model to remember; but on the downside, it means larger embedding matrices, which require more data to learn.

Overall, subword tokenization provides a way to easily scale between character tokenization (i.e., using a small subword vocab) and word tokenization (i.e., using a large subword vocab), and handles every human language without needing language-specific algorithms to be developed. It can even handle other "languages" such as genomic sequences or MIDI music notation! For this reason, in the last year its popularity has soared, and it seems likely to become the most common tokenization approach (it may well already be, by the time you read this!).

---

选择子词表的大小是一种折衷方案：较大的词表意味着每个句子中的词块较少，这意味着训练速度更快、内存更少、模型需要记忆的状态更少；但不利的一面是，这意味着嵌入矩阵更大，需要更多的数据来学习。

总的来说，子词标记化提供了一种在字符标记化（即使用小的子词词汇表）和单词标记化（即使用大的子词词汇表）之间轻松扩展的方法，并且可以处理每一种人类语言，而无需开发特定语言的算法。它甚至可以处理其他 "语言"，如基因组序列或 MIDI 音乐符号！因此，在过去的一年中，它的受欢迎程度急剧上升，似乎很有可能成为最常用的标记化方法（当您读到这篇文章时，它很可能已经是了！）。

Once our texts have been split into tokens, we need to convert them to numbers. We'll look at that next.

---

一旦我们的文本被分割成标记，我们就需要将它们转换成数字。我们接下来要看看这一点。

### Numericalization with fastai

*Numericalization* is the process of mapping tokens to integers. The steps are basically identical to those necessary to create a `Category` variable, such as the dependent variable of digits in MNIST:

1. Make a list of all possible levels of that categorical variable (the vocab).
1. Replace each level with its index in the vocab.

Let's take a look at this in action on the word-tokenized text we saw earlier:

---

*数值化*是将标记映射为整数的过程。其步骤与创建 `Category` 变量（如 MNIST 中的数字因变量）所需的步骤基本相同：

1. 列出该分类变量（词汇表）的所有可能级别。
1. 用词汇表中的索引替换每个级别。

让我们在前面看到的单词标记文本中看看这个方法的实际应用：

In [ ]:
toks = tkn(txt)
print(coll_repr(tkn(txt), 31))

(#228) ['xxbos','xxmaj','this','movie',',','which','i','just','discovered','at','the','video','store',',','has','apparently','sit','around','for','a','couple','of','years','without','a','distributor','.','xxmaj','it',"'s",'easy'...]


Just like with `SubwordTokenizer`, we need to call `setup` on `Numericalize`; this is how we create the vocab. That means we'll need our tokenized corpus first. Since tokenization takes a while, it's done in parallel by fastai; but for this manual walkthrough, we'll use a small subset:

---

与 `SubwordTokenizer` 一样，我们需要在 `Numericalize` 上调用 `setup` ；这就是我们创建词汇表的方法。这意味着我们首先需要标记化的语料库。由于标记化需要一段时间，因此 fastai 会并行处理；但在本手册中，我们将使用一小部分：

In [ ]:
toks200 = txts[:200].map(tkn)
toks200[0]

(#228) ['xxbos','xxmaj','this','movie',',','which','i','just','discovered','at'...]

We can pass this to `setup` to create our vocab:

---

我们可以将其传递给 `setup` 来创建我们的词汇表：

In [ ]:
num = Numericalize()
num.setup(toks200)
coll_repr(num.vocab,20)

"(#2000) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','.',',','a','and','of','to','is','in','i','it'...]"

Our special rules tokens appear first, and then every word appears once, in frequency order. The defaults to `Numericalize` are `min_freq=3,max_vocab=60000`. `max_vocab=60000` results in fastai replacing all words other than the most common 60,000 with a special *unknown word* token, `xxunk`. This is useful to avoid having an overly large embedding matrix, since that can slow down training and use up too much memory, and can also mean that there isn't enough data to train useful representations for rare words. However, this last issue is better handled by setting `min_freq`; the default `min_freq=3` means that any word appearing less than three times is replaced with `xxunk`.

fastai can also numericalize your dataset using a vocab that you provide, by passing a list of words as the `vocab` parameter.

Once we've created our `Numericalize` object, we can use it as if it were a function:

---

我们的特殊规则标记会首先出现，然后每个单词都会按频率顺序出现一次。`Numericalize` 的默认值是 `min_freq=3,max_vocab=60000`。`max_vocab=60000` 会使 fastai 用一个特殊的 *unknown word* token: `xxxunk` 来替换最常见的 60,000 个单词以外的所有单词。这有助于避免嵌入矩阵过大，因为这会减慢训练速度并占用过多内存，同时也意味着没有足够的数据来训练有用的稀有词表示。不过，通过设置 `min_freq` 可以更好地处理最后一个问题；默认的 `min_freq=3` 意味着任何出现次数少于三次的词都会被替换为 `xxxunk`。

fastai 还可以使用你提供的词汇表对数据集进行数值化，方法是将词汇列表作为 `vocab` 参数传递。

创建了 `Numericalize` 对象后，我们就可以像使用函数一样使用它了：

In [ ]:
nums = num(toks)[:20]; nums

tensor([  2,   8,  21,  28,  11,  90,  18,  59,   0,  45,   9, 351, 499,  11,  72, 533, 584, 146,  29,  12])

This time, our tokens have been converted to a tensor of integers that our model can receive. We can check that they map back to the original text:

---

这一次，我们的标记已被转换成我们的模型可以接收的整数张量。我们可以检查它们是否映射回原始文本：

In [ ]:
' '.join(num.vocab[o] for o in nums)

'xxbos xxmaj this movie , which i just xxunk at the video store , has apparently sit around for a'

Now that we have numbers, we need to put them in batches for our model.

---

现在，我们已经有了数字，我们需要将它们放入模型的小批量中。

### Putting Our Texts into Batches for a Language Model

When dealing with images, we needed to resize them all to the same height and width before grouping them together in a mini-batch so they could stack together efficiently in a single tensor. Here it's going to be a little different, because one cannot simply resize text to a desired length. Also, we want our language model to read text in order, so that it can efficiently predict what the next word is. This means that each new batch should begin precisely where the previous one left off.

Suppose we have the following text:

> : In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while.

The tokenization process will add special tokens and deal with punctuation to return this text:

> : xxbos xxmaj in this chapter , we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface . xxmaj first we will look at the processing steps necessary to convert text into numbers and how to customize it . xxmaj by doing this , we 'll have another example of the preprocessor used in the data block xxup api . \n xxmaj then we will study how we build a language model and train it for a while .

We now have 90 tokens, separated by spaces. Let's say we want a batch size of 6. We need to break this text into 6 contiguous parts of length 15:

---

在处理图片时，我们需要先将所有图片调整到相同的高度和宽度，然后再将它们集中到一个小批量中，这样它们就可以有效地堆叠在一个张量中。在这里，情况会有些不同，因为我们不能简单地将文本调整到所需的长度。此外，我们希望语言模型能按顺序读取文本，这样它就能有效预测下一个单词是什么。这就意味着，每一批新的文本都应从上一批文本结束的地方开始。

假设我们有以下文本：

> 在本章中，我们将回顾第一章中学习的电影评论分类示例，并进行深入挖掘。首先，我们将了解将文本转换为数字所需的处理步骤，以及如何对其进行自定义。然后，我们将研究如何建立语言模型并对其进行一段时间的训练。

标记化过程将添加特殊标记并处理标点符号，从而返回文本：

> xxmaj 首先，我们将看看将文本转换为数字所需的处理步骤以及如何对其进行自定义。xxmaj 通过这样做，我们将有另一个在数据块 xxup api 中使用预处理器的例子。\xxmaj 然后我们将研究如何构建一个语言模型并对其进行一段时间的训练。

现在我们有 90 个以空格分隔的标记。我们需要将这些文本分成长度为 15 的 6 个连续部分：

In [ ]:
#hide_input
stream = "In this chapter, we will go back over the example of classifying movie reviews we studied in chapter 1 and dig deeper under the surface. First we will look at the processing steps necessary to convert text into numbers and how to customize it. By doing this, we'll have another example of the PreProcessor used in the data block API.\nThen we will study how we build a language model and train it for a while."
tokens = tkn(stream)
bs,seq_len = 6,15
d_tokens = np.array([tokens[i*seq_len:(i+1)*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter,",",we,will,go,back,over,the,example,of,classifying
movie,reviews,we,studied,in,chapter,1,and,dig,deeper,under,the,surface,.,xxmaj
first,we,will,look,at,the,processing,steps,necessary,to,convert,text,into,numbers,and
how,to,customize,it,.,xxmaj,by,doing,this,",",we,'ll,have,another,example
of,the,preprocessor,used,in,the,data,block,xxup,api,.,\n,xxmaj,then,we
will,study,how,we,build,a,language,model,and,train,it,for,a,while,.


In a perfect world, we could then give this one batch to our model. But that approach doesn't scale, because outside of this toy example it's unlikely that a single batch containing all the texts would fit in our GPU memory (here we have 90 tokens, but all the IMDb reviews together give several million).

So, we need to divide this array more finely into subarrays of a fixed sequence length. It is important to maintain order within and across these subarrays, because we will use a model that maintains a state so that it remembers what it read previously when predicting what comes next. 

Going back to our previous example with 6 batches of length 15, if we chose a sequence length of 5, that would mean we first feed the following array:

---

在一个完美的世界里，我们可以将这一批文本交给我们的模型。但这种方法无法扩展，因为在这个玩具例子之外，我们的 GPU 内存不太可能容纳一个包含所有文本的批次（这里我们有 90 个标记，但所有 IMDb 评论加起来有几百万个）。

因此，我们需要将这个数组更精细地划分为具有固定序列长度的子数组。在这些子数组内部和之间保持顺序非常重要，因为我们将使用一个保持状态的模型，这样它在预测下一步时就能记住之前读取的内容。

回到我们之前的例子，6 个批次的长度都是 15，如果我们选择的序列长度是 5，这意味着我们首先要输入以下数组：

In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15:i*15+seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

xxbos,xxmaj,in,this,chapter
movie,reviews,we,studied,in
first,we,will,look,at
how,to,customize,it,.
of,the,preprocessor,used,in
will,study,how,we,build


Then this one:

In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+seq_len:i*15+2*seq_len] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

",",we,will,go,back
chapter,1,and,dig,deeper
the,processing,steps,necessary,to
xxmaj,by,doing,this,","
the,data,block,xxup,api
a,language,model,and,train


And finally:

In [ ]:
#hide_input
bs,seq_len = 6,5
d_tokens = np.array([tokens[i*15+10:i*15+15] for i in range(bs)])
df = pd.DataFrame(d_tokens)
display(HTML(df.to_html(index=False,header=None)))

over,the,example,of,classifying
under,the,surface,.,xxmaj
convert,text,into,numbers,and
we,'ll,have,another,example
.,\n,xxmaj,then,we
it,for,a,while,.


Going back to our movie reviews dataset, the first step is to transform the individual texts into a stream by concatenating them together. As with images, it's best to randomize the order of the inputs, so at the beginning of each epoch we will shuffle the entries to make a new stream (we shuffle the order of the documents, not the order of the words inside them, or the texts would not make sense anymore!).

We then cut this stream into a certain number of batches (which is our *batch size*). For instance, if the stream has 50,000 tokens and we set a batch size of 10, this will give us 10 mini-streams of 5,000 tokens. What is important is that we preserve the order of the tokens (so from 1 to 5,000 for the first mini-stream, then from 5,001 to 10,000...), because we want the model to read continuous rows of text (as in the preceding example). An `xxbos` token is added at the start of each during preprocessing, so that the model knows when it reads the stream when a new entry is beginning.

So to recap, at every epoch we shuffle our collection of documents and concatenate them into a stream of tokens. We then cut that stream into a batch of fixed-size consecutive mini-streams. Our model will then read the mini-streams in order, and thanks to an inner state, it will produce the same activation whatever sequence length we picked.

This is all done behind the scenes by the fastai library when we create an `LMDataLoader`. We do this by first applying our `Numericalize` object to the tokenized texts:

---

回到我们的影评数据集，第一步是将单个文本串联起来，转换成一个数据流。与处理图像一样，最好是随机调整输入的顺序，因此在每个周期开始时，我们将对条目进行洗牌，以生成新的数据流（我们洗牌的是文档的顺序，而不是文档中单词的顺序，否则文本将不再有意义！）。

然后，我们将这个流切成一定数量的批次（这就是我们的 *批次大小* ）。例如，如果数据流中有 50,000 个词组，而我们设定的批量大小为 10，那么就会得到 10 个 5,000 个词组的迷你数据流。重要的是，我们要保持标记的顺序（第一个迷你流从 1 到 5000，然后从 5001 到 10000......），因为我们希望模型读取连续行的文本（如前面的例子）。在预处理过程中，我们会在每条数据流的开头添加一个 `xxbos` 标记，以便模型在读取数据流时知道新的条目正在开始。

因此，概括地说，在每个时间点，我们都会对文档集合进行洗牌，并将它们串联成一个标记流。然后，我们将该流切割成一批固定大小的连续迷你流。然后，我们的模型会按顺序读取这些迷你流，由于有了内部状态，无论我们选择的序列长度是多少，它都会产生相同的激活。

当我们创建 `LMDataLoader ` 时，fastai 库会在幕后完成这一切。我们首先将 `Numericalize` 对象应用于标记化文本：

In [ ]:
nums200 = toks200.map(num)

and then passing that to `LMDataLoader`:

---

然后传入 `LMDataLoader`：

In [ ]:
dl = LMDataLoader(nums200)

Let's confirm that this gives the expected results, by grabbing the first batch:

---

让我们确认一下，这是不是产生了预期的结果，通过抓取第一个批次：

In [ ]:
x,y = first(dl)
x.shape,y.shape

(torch.Size([64, 72]), torch.Size([64, 72]))

and then looking at the first row of the independent variable, which should be the start of the first text:

---

然后看一下自变量的第一行，这应该是第一个文本的开始：

In [ ]:
' '.join(num.vocab[o] for o in x[0][:20])

'xxbos xxmaj this movie , which i just xxunk at the video store , has apparently sit around for a'

The dependent variable is the same thing offset by one token:

---

应变量是偏移一个 token 的相同东西：

In [ ]:
' '.join(num.vocab[o] for o in y[0][:20])

'xxmaj this movie , which i just xxunk at the video store , has apparently sit around for a couple'

This concludes all the preprocessing steps we need to apply to our data. We are now ready to train our text classifier.

---

这涵盖了我们需要对我们的数据使用的所有预处理步骤。我们现在准备训练我们的文本分类器。

## Training a Text Classifier

As we saw at the beginning of this chapter, there are two steps to training a state-of-the-art text classifier using transfer learning: first we need to fine-tune our language model pretrained on Wikipedia to the corpus of IMDb reviews, and then we can use that model to train a classifier.

As usual, let's start with assembling our data.

---

正如我们在本章的开头所看到的，使用迁移学习训练一个最先进的文本分类器需要两个步骤：首先，我们需要微调我们预先在维基百科上训练的语言模型，以适应IMDb评论的语料库，然后我们可以使用该模型来训练一个分类器。

像往常一样，让我们从组装我们的数据开始。

### Language Model Using DataBlock

fastai handles tokenization and numericalization automatically when `TextBlock` is passed to `DataBlock`. All of the arguments that can be passed to `Tokenize` and `Numericalize` can also be passed to `TextBlock`. In the next chapter we'll discuss the easiest ways to run each of these steps separately, to ease debugging—but you can always just debug by running them manually on a subset of your data as shown in the previous sections. And don't forget about `DataBlock`'s handy `summary` method, which is very useful for debugging data issues.

Here's how we use `TextBlock` to create a language model, using fastai's defaults:

---

当 `TextBlock` 传递给 `DataBlock` 时，fastai 会自动处理标记化和数值化。所有可以传递给 `Tokenize` 和 `Numericalize` 的参数也可以传递给 `TextBlock`。在下一章中，我们将讨论分别运行这些步骤的最简单方法，以方便调试，但你也可以像前面几节所示，在数据子集中手动运行这些步骤进行调试。不要忘记 `DataBlock` 的便捷 `summary` 方法，它对调试数据问题非常有用。

下面是我们如何使用 `TextBlock` 创建语言模型，使用 fastai 的默认值：

In [ ]:
get_imdb = partial(get_text_files, folders=['train', 'test', 'unsup'])

dls_lm = DataBlock(
    blocks=TextBlock.from_folder(path, is_lm=True),
    get_items=get_imdb, splitter=RandomSplitter(0.1)
).dataloaders(path, path=path, bs=128, seq_len=80)

One thing that's different to previous types we've used in `DataBlock` is that we're not just using the class directly (i.e., `TextBlock(...)`, but instead are calling a *class method*. A class method is a Python method that, as the name suggests, belongs to a *class* rather than an *object*. (Be sure to search online for more information about class methods if you're not familiar with them, since they're commonly used in many Python libraries and applications; we've used them a few times previously in the book, but haven't called attention to them.) The reason that `TextBlock` is special is that setting up the numericalizer's vocab can take a long time (we have to read and tokenize every document to get the vocab). To be as efficient as possible it performs a few optimizations: 

- It saves the tokenized documents in a temporary folder, so it doesn't have to tokenize them more than once
- It runs multiple tokenization processes in parallel, to take advantage of your computer's CPUs

We need to tell `TextBlock` how to access the texts, so that it can do this initial preprocessing—that's what `from_folder` does.

`show_batch` then works in the usual way:

---

与我们之前在 `DataBlock` 中使用的类型不同的一点是，我们不是直接使用类（即 `TextBlock(...)` ，而是调用一个 *类方法* 。类方法是一种 Python 方法，顾名思义，它属于一个 *类* ，而不是一个 *对象* 。(如果您不熟悉类方法，请务必上网搜索有关类方法的更多信息，因为它们在许多 Python 库和应用程序中都很常用；我们在书中已经使用过几次，但没有引起注意)。之所以说 `TextBlock` 很特别，是因为设置数值化器的词汇表可能需要很长时间（我们必须读取并标记每个文档以获取词汇表）。为了尽可能提高效率，它进行了一些优化：

- 将标记化后的文档保存在一个临时文件夹中，这样就不必多次标记化文档了
- 并行运行多个标记化进程，以充分利用电脑的 CPU

我们需要告诉 `TextBlock` 如何访问文本，这样它才能进行初始预处理--这就是 `from_folder` 的作用。

然后，`show_batch` 会以通常的方式工作：

In [ ]:
dls_lm.show_batch(max_n=2)

,text,text_
0,"xxbos xxmaj it 's awesome ! xxmaj in xxmaj story xxmaj mode , your going from punk to pro . xxmaj you have to complete goals that involve skating , driving , and walking . xxmaj you create your own skater and give it a name , and you can make it look stupid or realistic . xxmaj you are with your friend xxmaj eric throughout the game until he betrays you and gets you kicked off of the skateboard","xxmaj it 's awesome ! xxmaj in xxmaj story xxmaj mode , your going from punk to pro . xxmaj you have to complete goals that involve skating , driving , and walking . xxmaj you create your own skater and give it a name , and you can make it look stupid or realistic . xxmaj you are with your friend xxmaj eric throughout the game until he betrays you and gets you kicked off of the skateboard xxunk"
1,"what xxmaj i 've read , xxmaj death xxmaj bed is based on an actual dream , xxmaj george xxmaj barry , the director , successfully transferred dream to film , only a genius could accomplish such a task . \n\n xxmaj old mansions make for good quality horror , as do portraits , not sure what to make of the killer bed with its killer yellow liquid , quite a bizarre dream , indeed . xxmaj also , this","xxmaj i 've read , xxmaj death xxmaj bed is based on an actual dream , xxmaj george xxmaj barry , the director , successfully transferred dream to film , only a genius could accomplish such a task . \n\n xxmaj old mansions make for good quality horror , as do portraits , not sure what to make of the killer bed with its killer yellow liquid , quite a bizarre dream , indeed . xxmaj also , this is"


Now that our data is ready, we can fine-tune the pretrained language model.

---

现在我们的数据准备好了，我们可以微调预先训练好的语言模型。

### Fine-Tuning the Language Model

To convert the integer word indices into activations that we can use for our neural network, we will use embeddings, just like we did for collaborative filtering and tabular modeling. Then we'll feed those embeddings into a *recurrent neural network* (RNN), using an architecture called *AWD-LSTM* (we will show you how to write such a model from scratch in <<chapter_nlp_dive>>). As we discussed earlier, the embeddings in the pretrained model are merged with random embeddings added for words that weren't in the pretraining vocabulary. This is handled automatically inside `language_model_learner`:

---

为了将整数词索引转换为神经网络可以使用的激活，我们将使用嵌入，就像我们在协同过滤和表格建模中所做的那样。然后，我们将使用一种名为 *AWD-LSTM* 的体系结构，将这些嵌入值送入*递归神经网络*（RNN）（我们将在<<chapter_nlp_dive>>中向您展示如何从头开始编写这样一个模型）。正如我们之前所讨论的，预训练模型中的嵌入与随机添加的嵌入合并在一起，用于预训练词汇表中没有的单词。这将在 `language_model_learner` 中自动处理：

In [ ]:
learn = language_model_learner(
    dls_lm, AWD_LSTM, drop_mult=0.3, 
    metrics=[accuracy, Perplexity()]).to_fp16()

The loss function used by default is cross-entropy loss, since we essentially have a classification problem (the different categories being the words in our vocab). The *perplexity* metric used here is often used in NLP for language models: it is the exponential of the loss (i.e., `torch.exp(cross_entropy)`). We  also include the accuracy metric, to see how many times our model is right when trying to predict the next word, since cross-entropy (as we've seen) is both hard to interpret, and tells us more about the model's confidence than its accuracy.

Let's go back to the process diagram from the beginning of this chapter. The first arrow has been completed for us and made available as a pretrained model in fastai, and we've just built the `DataLoaders` and `Learner` for the second stage. Now we're ready to fine-tune our language model!

---

默认情况下使用的损失函数是交叉熵损失，因为我们基本上面临的是一个分类问题（不同的类别就是词汇表中的单词）。这里使用的 *perplexity* 指标通常用于 NLP 中的语言模型：它是损失的指数（即 `torch.exp(cross_entropy)`）。由于交叉熵（我们已经看到）很难解释，而且它告诉我们的更多是模型的置信度而非准确度，因此我们还加入了准确度指标，以了解我们的模型在尝试预测下一个词时有多少次是正确的。

让我们回到本章开头的流程图。第一支箭已经完成，并在 fastai 中作为预训练模型提供，我们刚刚为第二阶段构建了 `DataLoaders` 和 `Learner`。现在，我们准备对语言模型进行微调！

<img alt="Diagram of the ULMFiT process" width="450" src="images/att_00027.png">

It takes quite a while to train each epoch, so we'll be saving the intermediate model results during the training process. Since `fine_tune` doesn't do that for us, we'll use `fit_one_cycle`. Just like `vision_learner`, `language_model_learner` automatically calls `freeze` when using a pretrained model (which is the default), so this will only train the embeddings (the only part of the model that contains randomly initialized weights—i.e., embeddings for words that are in our IMDb vocab, but aren't in the pretrained model vocab):

---

训练每个 epoch 需要相当长的时间，因此我们将在训练过程中保存模型的中间结果。由于 `fine_tune` 无法为我们做到这一点，我们将使用 `fit_one_cycle`。与 `vision_learner` 一样，当使用预训练模型时，`language_model_learner` 也会自动调用 `freeze`（冻结）（这是默认设置），因此这只会训练嵌入（模型中唯一包含随机初始化权重的部分，即 IMDb 词汇表中的单词嵌入，但不在预训练模型词汇表中）：

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.120048,3.912788,0.299565,50.038246,11:39


This model takes a while to train, so it's a good opportunity to talk about saving intermediary results. 

---

这个模型训练需要一段时间，所以这是一个很好的机会来谈谈保存中间结果。

### Saving and Loading Models

You can easily save the state of your model like so:

---

你可以像这样轻松地保存模型的状态：

In [ ]:
learn.save('1epoch')

This will create a file in `learn.path/models/` named *1epoch.pth*. If you want to load your model in another machine after creating your `Learner` the same way, or resume training later, you can load the content of this file with:

---

这会在 `learn.path/models/` 中创建一个名为 *1epoch.pth* 的文件。如果你想在你创建 `Learner` 之后在其他机器上加载你的模型，或者稍后恢复训练，你可以用下面的方法加载这个文件的内容：

In [ ]:
learn = learn.load('1epoch')

Once the initial training has completed, we can continue fine-tuning the model after unfreezing:

---

一旦初始训练完成，我们可以在解冻模型后继续微调模型：

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, 2e-3)

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.893486,3.772820,0.317104,43.502548,12:37
1,3.820479,3.717197,0.323790,41.148880,12:30
2,3.735622,3.659760,0.330321,38.851997,12:09
3,3.677086,3.624794,0.333960,37.516987,12:12
4,3.636646,3.601300,0.337017,36.645859,12:05
5,3.553636,3.584241,0.339355,36.026001,12:04
6,3.507634,3.571892,0.341353,35.583862,12:08
7,3.444101,3.565988,0.342194,35.374371,12:08
8,3.398597,3.566283,0.342647,35.384815,12:11
9,3.375563,3.568166,0.342528,35.451500,12:05


Once this is done, we save all of our model except the final layer that converts activations to probabilities of picking each token in our vocabulary. The model not including the final layer is called the *encoder*. We can save it with `save_encoder`:

---

完成后，我们将保存模型的全部内容，但最后一层除外，该层将激活转换为选取词汇表中每个 token 的概率。不包括最后一层的模型称为 *编码器* 。我们可以用 `save_encoder` 保存它：

In [ ]:
learn.save_encoder('finetuned')

> jargon: Encoder: The model not including the task-specific final layer(s). This term means much the same thing as _body_ when applied to vision CNNs, but "encoder" tends to be more used for NLP and generative models.

---

> 行话：编码器：不包括特定任务最终层的模型。当应用于视觉 CNN 时，该术语的含义与 _body_ 大致相同，但 "编码器(encoder)"往往更多用于 NLP 和生成模型。

This completes the second stage of the text classification process: fine-tuning the language model. We can now use it to fine-tune a classifier using the IMDb sentiment labels.

---

这就完成了文本分类过程的第二阶段：微调语言模型。现在，我们可以使用它来微调使用 IMDb 情感标签的分类器。

### Text Generation

Before we move on to fine-tuning the classifier, let's quickly try something different: using our model to generate random reviews. Since it's trained to guess what the next word of the sentence is, we can use the model to write new reviews:

---

在对分类器进行微调之前，让我们快速尝试一些不同的方法：使用我们的模型生成随机评论。由于模型经过训练可以猜测句子的下一个单词是什么，因此我们可以使用模型编写新的评论：

In [ ]:
TEXT = "I liked this movie because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

In [ ]:
print("\n".join(preds))

i liked this movie because of its story and characters . The story line was very strong , very good for a sci - fi film . The main character , Alucard , was very well developed and brought the whole story
i liked this movie because i like the idea of the premise of the movie , the ( very ) convenient virus ( which , when you have to kill a few people , the " evil " machine has to be used to protect


As you can see, we add some randomness (we pick a random word based on the probabilities returned by the model) so we don't get exactly the same review twice. Our model doesn't have any programmed knowledge of the structure of a sentence or grammar rules, yet it has clearly learned a lot about English sentences: we can see it capitalizes properly (*I* is just transformed to *i* because our rules require two characters or more to consider a word as capitalized, so it's normal to see it lowercased) and is using consistent tense. The general review makes sense at first glance, and it's only if you read carefully that you can notice something is a bit off. Not bad for a model trained in a couple of hours! 

But our end goal wasn't to train a model to generate reviews, but to classify them... so let's use this model to do just that.

---

正如你所看到的，我们添加了一些随机性（我们根据模型返回的概率随机选择一个单词），这样我们就不会两次得到完全相同的评论。我们的模型没有任何关于句子结构或语法规则的编程知识，但它显然已经学习了很多关于英语句子的知识：我们可以看到它正确地进行了大写（*I* 只是转换成了*i*，因为我们的规则要求两个或两个以上字符才能将一个单词视为大写，所以看到它小写是正常的），并且使用了一致的时态。总的评论乍一看很有道理，只有仔细阅读才能发现有些地方不对。对于几个小时内训练出来的模型来说，这还算不错！

但我们的最终目标并不是训练一个模型来生成评论，而是对评论进行分类......因此，让我们用这个模型来做这件事。

### Creating the Classifier DataLoaders

We're now moving from language model fine-tuning to classifier fine-tuning. To recap, a language model predicts the next word of a document, so it doesn't need any external labels. A classifier, however, predicts some external label—in the case of IMDb, it's the sentiment of a document.

This means that the structure of our `DataBlock` for NLP classification will look very familiar. It's actually nearly the same as we've seen for the many image classification datasets we've worked with:

---

我们现在要从语言模型微调转向分类器微调。概括地说，语言模型预测文档的下一个单词，因此它不需要任何外部标签。而分类器则会预测一些外部标签--在 IMDb 的例子中，它预测的是文档的情感。

这意味着我们用于 NLP 分类的 "数据块 "结构看起来非常熟悉。实际上，它与我们在许多图像分类数据集上看到的几乎一样：

In [ ]:
dls_clas = DataBlock(
    blocks=(TextBlock.from_folder(path, vocab=dls_lm.vocab),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path, path=path, bs=128, seq_len=72)

Just like with image classification, `show_batch` shows the dependent variable (sentiment, in this case) with each independent variable (movie review text):

---

与图像分类一样， `show_batch` 会显示因变量（本例中为情感）和每个自变量（影评文本）：

In [ ]:
dls_clas.show_batch(max_n=3)

,text,category
0,"xxbos i rate this movie with 3 skulls , only coz the girls knew how to scream , this could 've been a better movie , if actors were better , the twins were xxup ok , i believed they were evil , but the eldest and youngest brother , they sucked really bad , it seemed like they were reading the scripts instead of acting them … . spoiler : if they 're vampire 's why do they freeze the blood ? vampires ca n't drink frozen blood , the sister in the movie says let 's drink her while she is alive … .but then when they 're moving to another house , they take on a cooler they 're frozen blood . end of spoiler \n\n it was a huge waste of time , and that made me mad coz i read all the reviews of how",neg
1,"xxbos i have read all of the xxmaj love xxmaj come xxmaj softly books . xxmaj knowing full well that movies can not use all aspects of the book , but generally they at least have the main point of the book . i was highly disappointed in this movie . xxmaj the only thing that they have in this movie that is in the book is that xxmaj missy 's father comes to xxunk in the book both parents come ) . xxmaj that is all . xxmaj the story line was so twisted and far fetch and yes , sad , from the book , that i just could n't enjoy it . xxmaj even if i did n't read the book it was too sad . i do know that xxmaj pioneer life was rough , but the whole movie was a downer . xxmaj the rating",neg
2,"xxbos xxmaj this , for lack of a better term , movie is lousy . xxmaj where do i start … … \n\n xxmaj cinemaphotography - xxmaj this was , perhaps , the worst xxmaj i 've seen this year . xxmaj it looked like the camera was being tossed from camera man to camera man . xxmaj maybe they only had one camera . xxmaj it gives you the sensation of being a volleyball . \n\n xxmaj there are a bunch of scenes , haphazardly , thrown in with no continuity at all . xxmaj when they did the ' split screen ' , it was absurd . xxmaj everything was squished flat , it looked ridiculous . \n\n xxmaj the color tones were way off . xxmaj these people need to learn how to balance a camera . xxmaj this ' movie ' is poorly made , and",neg


Looking at the `DataBlock` definition, every piece is familiar from previous data blocks we've built, with two important exceptions:

- `TextBlock.from_folder` no longer has the `is_lm=True` parameter.
- We pass the `vocab` we created for the language model fine-tuning.

The reason that we pass the `vocab` of the language model is to make sure we use the same correspondence of token to index. Otherwise the embeddings we learned in our fine-tuned language model won't make any sense to this model, and the fine-tuning step won't be of any use.

By passing `is_lm=False` (or not passing `is_lm` at all, since it defaults to `False`) we tell `TextBlock` that we have regular labeled data, rather than using the next tokens as labels. There is one challenge we have to deal with, however, which is to do with collating multiple documents into a mini-batch. Let's see with an example, by trying to create a mini-batch containing the first 10 documents. First we'll numericalize them:

---

从 `DataBlock` 的定义来看，除了两个重要的例外，每一部分都与我们以前构建的数据块非常相似：

- `TextBlock.from_folder` 不再有 `is_lm=True` 参数。
- 我们传递为语言模型微调创建的 `vocab` 参数。

我们传递语言模型的 `vocab` 的原因是确保我们使用相同的标记与索引的对应关系。否则，我们在微调后的语言模型中学习到的嵌入对这个模型就没有任何意义，微调步骤也就没有任何用处。

通过传递 `is_lm=False`（或者根本不传递 `is_lm`，因为它默认为 `False`），我们可以告诉 `TextBlock` 我们有正规的标签数据，而不是使用下一个标记作为标签。不过，我们还必须应对一个挑战，那就是将多个文档整理成一个小批次。让我们举个例子，尝试创建一个包含前 10 个文档的迷你批次。首先，我们将它们数值化：

In [ ]:
nums_samp = toks200[:10].map(num)

Let's now look at how many tokens each of these 10 movie reviews have:

---

让我们现在看一看这些 10 个电影评论有多少个 token ：

In [ ]:
nums_samp.map(len)

(#10) [228,238,121,290,196,194,533,124,581,155]

Remember, PyTorch `DataLoader`s need to collate all the items in a batch into a single tensor, and a single tensor has a fixed shape (i.e., it has some particular length on every axis, and all items must be consistent). This should sound familiar: we had the same issue with images. In that case, we used cropping, padding, and/or squishing to make all the inputs the same size. Cropping might not be a good idea for documents, because it seems likely we'd remove some key information (having said that, the same issue is true for images, and we use cropping there; data augmentation hasn't been well explored for NLP yet, so perhaps there are actually opportunities to use cropping in NLP too!). You can't really "squish" a document. So that leaves padding!

We will expand the shortest texts to make them all the same size. To do this, we use a special padding token that will be ignored by our model. Additionally, to avoid memory issues and improve performance, we will batch together texts that are roughly the same lengths (with some shuffling for the training set). We do this by (approximately, for the training set) sorting the documents by length prior to each epoch. The result of this is that the documents collated into a single batch will tend to be of similar lengths. We won't pad every batch to the same size, but will instead use the size of the largest document in each batch as the target size. (It is possible to do something similar with images, which is especially useful for irregularly sized rectangular images, but at the time of writing no library provides good support for this yet, and there aren't any papers covering it. It's something we're planning to add to fastai soon, however, so keep an eye on the book's website; we'll add information about this as soon as we have it working well.)

The sorting and padding are automatically done by the data block API for us when using a `TextBlock`, with `is_lm=False`. (We don't have this same issue for language model data, since we concatenate all the documents together first, and then split them into equally sized sections.)

We can now create a model to classify our texts:

---

请记住，PyTorch 的 `DataLoaders` 需要将批次中的所有项目整理成一个单一的张量，而一个单一的张量有固定的形状（即它在每个轴上都有特定的长度，并且所有项目必须一致）。这听起来很熟悉：我们在处理图像时也遇到过同样的问题。在这种情况下，我们使用裁剪、填充和/或挤压的方法使所有输入的大小相同。对于文档来说，裁剪可能不是一个好主意，因为我们很可能会删除一些关键信息（尽管如此，对于图像来说也存在同样的问题，所以我们也使用了裁剪的方法；对于 NLP 来说，数据增强还没有得到很好的探索，所以也许在 NLP 中也有使用裁剪的机会！）。你无法真正 "压扁" 文档。所以就剩下填充了！

我们将扩充最短的文本，使其大小相同。为此，我们将使用一个特殊的填充标记，该标记将被我们的模型忽略。此外，为了避免内存问题并提高性能，我们将把长度大致相同的文本集中在一起（在训练集中进行了一些调整）。我们的做法是（对于训练集而言，大致如此）在每次计时之前按长度对文档进行排序。这样做的结果是，整理成一个批次的文档往往长度相似。我们不会将每批文档填充为相同大小，而是将每批文档中最大文档的大小作为目标大小。（对图像也可以做类似的处理，这对不规则尺寸的矩形图像尤其有用，但在撰写本文时，还没有任何库对此提供良好的支持，也没有任何论文对此有所涉及。不过，我们计划很快将其添加到 fastai 中，所以请随时关注本书的网站；一旦运行良好，我们将尽快添加相关信息）。

在使用 `is_lm=False` 的 `TextBlock` 时，data block API 会自动为我们进行排序和填充。（对于语言模型数据，我们不存在同样的问题，因为我们会先将所有文档连接在一起，然后再将它们分割成大小相等的部分）。

现在我们可以创建一个模型来对文本进行分类：

In [ ]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, 
                                metrics=accuracy).to_fp16()

The final step prior to training the classifier is to load the encoder from our fine-tuned language model. We use `load_encoder` instead of `load` because we only have pretrained weights available for the encoder; `load` by default raises an exception if an incomplete model is loaded:

---

训练分类器前的最后一步是从微调语言模型中加载编码器。我们使用 `load_encoder` 而不是 `load`，是因为我们只为编码器提供了预训练的权重；如果加载了不完整的模型，`load` 默认情况下会引发异常：

In [ ]:
learn = learn.load_encoder('finetuned')

### Fine-Tuning the Classifier

The last step is to train with discriminative learning rates and *gradual unfreezing*. In computer vision we often unfreeze the model all at once, but for NLP classifiers, we find that unfreezing a few layers at a time makes a real difference:

---

最后一步是使用辨别学习率和 *渐进解冻* 进行训练。在计算机视觉中，我们通常会一次性解冻模型，但对于 NLP 分类器，我们发现每次解冻几层会带来真正的不同：

In [ ]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.347427,0.184480,0.929320,00:33


In just one epoch we get the same result as our training in <<chapter_intro>>: not too bad! We can pass `-2` to `freeze_to` to freeze all except the last two parameter groups:

---

只用了一个 epoch ，我们就得到了与 <<chapter_intro>> 中的训练相同的结果：还不赖！我们可以向 `freeze_to` 传递 `-2` 以冻结除最后两个参数组之外的所有参数：

In [ ]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,0.247763,0.171683,0.934640,00:37


Then we can unfreeze a bit more, and continue training:

In [ ]:
learn.freeze_to(-3)
learn.fit_one_cycle(1, slice(5e-3/(2.6**4),5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.193377,0.156696,0.941200,00:45


And finally, the whole model!

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.172888,0.153770,0.943120,01:01
1,0.161492,0.155567,0.942640,00:57


We reached 94.3% accuracy, which was state-of-the-art performance just three years ago. By training another model on all the texts read backwards and averaging the predictions of those two models, we can even get to 95.1% accuracy, which was the state of the art introduced by the ULMFiT paper. It was only beaten a few months ago, by fine-tuning a much bigger model and using expensive data augmentation techniques (translating sentences in another language and back, using another model for translation).

Using a pretrained model let us build a fine-tuned language model that was pretty powerful, to either generate fake reviews or help classify them. This is exciting stuff, but it's good to remember that this technology can also be used for malign purposes.

---

我们的准确率达到了 94.3%，这在三年前还是最先进的水平。通过在所有反向阅读文本上训练另一个模型，并对这两个模型的预测结果取平均值，我们甚至可以达到 95.1%的准确率，而这正是 ULMFiT 论文所介绍的技术水平。几个月前，通过对一个更大的模型进行微调，并使用昂贵的数据增强技术（将句子翻译成另一种语言，再用另一个模型进行翻译），我们才打破了这一纪录。

通过使用预训练模型，我们建立了一个功能强大的微调语言模型，可以生成虚假评论或帮助对其进行分类。这是令人兴奋的事情，但要记住，这种技术也可能被用于恶意目的。

## Disinformation and Language Models

Even simple algorithms based on rules, before the days of widely available deep learning language models, could be used to create fraudulent accounts and try to influence policymakers. Jeff Kao, now a computational journalist at ProPublica, analyzed the comments that were sent to the US Federal Communications Commission (FCC) regarding a 2017 proposal to repeal net neutrality. In his article ["More than a Million Pro-Repeal Net Neutrality Comments Were Likely Faked"](https://hackernoon.com/more-than-a-million-pro-repeal-net-neutrality-comments-were-likely-faked-e9f0e3ed36a6), he reports how he discovered a large cluster of comments opposing net neutrality that seemed to have been generated by some sort of Mad Libs-style mail merge. In <<disinformation>>, the fake comments have been helpfully color-coded by Kao to highlight their formulaic nature.

---

在深度学习语言模型广泛应用之前，即使是基于规则的简单算法，也可能被用来创建欺诈性账户，并试图影响政策制定者。杰夫-高（Jeff Kao）现在是 ProPublica 的一名计算记者，他分析了就 2017 年废除网络中立性的提案向美国联邦通信委员会（FCC）提交的意见。在他的文章["一百多万条支持废除网络中立性的评论很可能是伪造的"](https://hackernoon.com/more-than-a-million-pro-repeal-net-neutrality-comments-were-likely-faked-e9f0e3ed36a6)中，他报告了自己是如何发现一大群反对网络中立性的评论的，这些评论似乎是通过某种 Mad Libs 式的邮件合并生成的。在 <<disinformation>> 中，Kao 对这些伪造的评论进行了有用的颜色编码，以突出它们的公式化性质。

<img src="images/ethics/image16.png" width="700" id="disinformation" caption="Comments received by the FCC during the net neutrality debate">

Kao estimated that "less than 800,000 of the 22M+ comments… could be considered truly unique" and that "more than 99% of the truly unique comments were in favor of keeping net neutrality."

Given advances in language modeling that have occurred since 2017, such fraudulent campaigns could be nearly impossible to catch now.  You now have all the necessary tools at your disposal to create a compelling language model—that is, something that can generate context-appropriate, believable text. It won't necessarily be perfectly accurate or correct, but it will be plausible. Think about what this technology would mean when put together with the kinds of disinformation campaigns we have learned about in recent years. Take a look at the Reddit dialogue shown in <<ethics_reddit>>, where a language model based on OpenAI's GPT-2 algorithm is having a conversation with itself about whether the US government should cut defense spending.

---

Kao 估计，"在 2200 多万条评论中，只有不到 80 万条......可被视为真正独特的评论"，"99% 以上真正独特的评论都支持保留网络中立性"。

鉴于 2017 年以来在语言建模方面取得的进步，现在几乎不可能抓住此类欺诈活动。  现在，您可以利用所有必要的工具来创建一个引人注目的语言模型，也就是说，可以生成与上下文相适应的、可信的文本。它不一定完全准确或正确，但一定可信。想一想，如果把这项技术与我们近年来了解到的各种虚假信息活动结合在一起，会意味着什么。看看 <<ethics_reddit>> 中显示的 Reddit 对话，基于 OpenAI 的 GPT-2 算法的语言模型正在就美国政府是否应该削减国防开支与自己进行对话。

<img src="images/ethics/image14.png" id="ethics_reddit" caption="An algorithm talking to itself on Reddit" alt="An algorithm talking to itself on Reddit" width="600">

In this case, it was explicitly said that an algorithm was used, but imagine what would happen if a bad actor decided to release such an algorithm across social networks. They could do it slowly and carefully, allowing the algorithm to gradually develop followers and trust over time. It would not take many resources to have literally millions of accounts doing this. In such a situation we could easily imagine getting to a point where the vast majority of discourse online was from bots, and nobody would have any idea that it was happening.

We are already starting to see examples of machine learning being used to generate identities. For example, <<katie_jones>> shows a LinkedIn profile for Katie Jones.

---

在这一案例中，明确表示使用的是一种算法，但试想一下，如果一个坏人决定在社交网络上发布这样一种算法，会发生什么。他们可以慢慢地、小心翼翼地进行，让算法随着时间的推移逐渐积累粉丝和信任。不需要太多的资源，就可以有数百万个账户这样做。在这种情况下，我们可以很容易地想象到，网络上的绝大多数言论都来自于机器人，而没有人会意识到这一点。

我们已经开始看到利用机器学习生成身份的例子。例如，<<katie_jones>> 显示的是凯蒂-琼斯（Katie Jones）的 LinkedIn 简介。

<img src="images/ethics/image15.jpeg" width="400" id="katie_jones" caption="Katie Jones's LinkedIn profile">

Katie Jones was connected on LinkedIn to several members of mainstream Washington think tanks. But she didn't exist. That image you see was auto-generated by a generative adversarial network, and somebody named Katie Jones has not, in fact, graduated from the Center for Strategic and International Studies.

Many people assume or hope that algorithms will come to our defense here—that we will develop classification algorithms that can automatically recognise autogenerated content. The problem, however, is that this will always be an arms race, in which better classification (or discriminator) algorithms can be used to create better generation algorithms.

---

凯蒂-琼斯（Katie Jones）在 LinkedIn 上与华盛顿主流智库的几位成员建立了联系。但她并不存在。你看到的那张图片是由对抗性网络自动生成的，事实上，一个叫凯蒂-琼斯的人并没有从战略与国际研究中心毕业。

很多人都认为或希望算法会在这里为我们辩护--我们会开发出能够自动识别自动生成内容的分类算法。但问题是，这永远是一场军备竞赛，在这场竞赛中，更好的分类（或鉴别）算法可以用来创造更好的生成算法。

## Conclusion

In this chapter we explored the last application covered out of the box by the fastai library: text. We saw two types of models: language models that can generate texts, and a classifier that determines if a review is positive or negative. To build a state-of-the art classifier, we used a pretrained language model, fine-tuned it to the corpus of our task, then used its body (the encoder) with a new head to do the classification.

Before we end this section, we'll take a look at how the fastai library can help you assemble your data for your specific problems.

---

在本章中，我们探讨了 fastai 库开箱即用的最后一个应用：文本。我们看到了两种类型的模型：一种是可以生成文本的语言模型，另一种是确定评论是正面还是负面的分类器。为了建立最先进的分类器，我们使用了一个预训练的语言模型，并根据我们任务的语料库对其进行了微调，然后使用其主体（编码器）和一个新的头部来进行分类。

在本节结束之前，我们将看看 fastai 库如何帮助您针对具体问题收集数据。

## Questionnaire

1. What is "self-supervised learning"?
1. What is a "language model"?
1. Why is a language model considered self-supervised?
1. What are self-supervised models usually used for?
1. Why do we fine-tune language models?
1. What are the three steps to create a state-of-the-art text classifier?
1. How do the 50,000 unlabeled movie reviews help us create a better text classifier for the IMDb dataset?
1. What are the three steps to prepare your data for a language model?
1. What is "tokenization"? Why do we need it?
1. Name three different approaches to tokenization.
1. What is `xxbos`?
1. List four rules that fastai applies to text during tokenization.
1. Why are repeated characters replaced with a token showing the number of repetitions and the character that's repeated?
1. What is "numericalization"?
1. Why might there be words that are replaced with the "unknown word" token?
1. With a batch size of 64, the first row of the tensor representing the first batch contains the first 64 tokens for the dataset. What does the second row of that tensor contain? What does the first row of the second batch contain? (Careful—students often get this one wrong! Be sure to check your answer on the book's website.)
1. Why do we need padding for text classification? Why don't we need it for language modeling?
1. What does an embedding matrix for NLP contain? What is its shape?
1. What is "perplexity"?
1. Why do we have to pass the vocabulary of the language model to the classifier data block?
1. What is "gradual unfreezing"?
1. Why is text generation always likely to be ahead of automatic identification of machine-generated texts?

---

1. 什么是 "自我监督学习"？
2. 什么是 "语言模型"？
3. 为什么语言模型被认为是自我监督的？
4. 自我监督模型通常用于哪些方面？
5. 为什么要对语言模型进行微调？
6. 创建最先进文本分类器的三个步骤是什么？
7. 50,000 篇未标注的电影评论如何帮助我们为 IMDb 数据集创建更好的文本分类器？
8. 为语言模型准备数据的三个步骤是什么？
9.  什么是 "标记化"？我们为什么需要它？
10. 说出标记化的三种不同方法。
11. 什么是 `xxbos`？
12. 列出 fastai 在标记化过程中应用于文本的四种规则。
13. 为什么重复的字符会被替换为显示重复次数和重复字符的标记？
14. 什么是 "数值化"？
15. 为什么有些词会被替换为 "未知词 "标记？
16. 如果批量大小为 64，代表第一批的张量的第一行包含数据集的前 64 个标记。该张量的第二行包含什么内容？第二批的第一行包含什么？(小心--学生们经常弄错这个问题！请务必在本书的网站上检查您的答案）。
17. 为什么文本分类需要填充？为什么语言建模不需要？
18. NLP 的嵌入矩阵包含哪些内容？它的形状是什么？
19. 什么是 "困惑度"？
20. 为什么要将语言模型的词汇传递给分类器数据块？
21. 什么是 "逐步解冻"？
22. 为什么文本生成总是可能领先于机器生成文本的自动识别？

### Further Research

1. See what you can learn about language models and disinformation. What are the best language models today? Take a look at some of their outputs. Do you find them convincing? How could a bad actor best use such a model to create conflict and uncertainty?
1. Given the limitation that models are unlikely to be able to consistently recognize machine-generated texts, what other approaches may be needed to handle large-scale disinformation campaigns that leverage deep learning?